In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

# PaLM on top of Vertex AI Search

## Search API

In [ ]:
#@title Search API

# ! pip install google-cloud-discoveryengine==0.11.0 --quiet
from google.cloud import discoveryengine_v1beta as discoveryengine
from google.protobuf.json_format import MessageToDict
from typing import List, Dict

def search_unstructured(PROJECT_ID, data):
    engine_id = data.get('engine_id')
    page_token = data.get('page_token')
    query = data.get('search_query')
    page_size = data.get('page_size')
    summary_result_count = data.get('summary_result_count')
    max_snippet_count = data.get('max_snippet_count')
    max_extractive_answer_count = data.get('max_extractive_answer_count')
    max_extractive_segment_count = data.get('max_extractive_segment_count')
    filter_id = data.get('filter_id')
    filter_facets = data.get('filter_facets')
    filter_tenant = data.get('filter_tenant')

    return search(
      PROJECT_ID,
      engine_id,
      page_token,
      query,
      page_size,
      summary_result_count,
      max_snippet_count,
      max_extractive_answer_count,
      max_extractive_segment_count,
      filter_id,
      filter_facets,
      filter_tenant
    )

def search(
    project_id,
    search_engine_id,
    page_token: str,
    search_query: str,
    page_size: int,
    summary_result_count: int,
    max_snippet_count: int,
    max_extractive_answer_count: int,
    max_extractive_segment_count: int,
    filter_id=List[str],
    filter_facets=Dict[str, List[str]],
    filter_tenant=str,

    facets=["category","tenant"]
    ):

    ## Create a client
    client = discoveryengine.SearchServiceClient()

    ## Argolis Parameters
    project_id = project_id
    search_engine_id = search_engine_id
    location = "global"
    serving_config_id = "default_config"
    search_query = search_query

    ## Config Path
    serving_config = client.serving_config_path(
        project=project_id,
        location=location,
        data_store=search_engine_id,
        serving_config=serving_config_id,
    )

    ## Content Search Spec
    content_search_spec = {
        'snippet_spec': { 'return_snippet': True },
        'extractive_content_spec': {
            'max_extractive_answer_count': max_extractive_answer_count,
            'max_extractive_segment_count' : max_extractive_segment_count
        },
        'summary_spec': {
            'summary_result_count' : summary_result_count,
            'include_citations': True,
        },
    }

    ## Facets
    facet_specs=[] ## if empty, no facets are returned (max=100)
    for facet in facets:
        facet_spec = {
            "facet_key": {
                "key": facet,
            },
            "limit": 20, ## Default=20, max facet values to be returned for this facet (max=300)
        }
        facet_specs.append(facet_spec)

    filter_parts = []
    if filter_id:
        filter_id_string = "id: ANY(\"" + "\", \"".join(filter_id) + "\")"
        filter_parts.append(filter_id_string)
    if filter_facets:
        category_filters = filter_facets.get('category', [])
        if category_filters:
            category_filter_string = "category: ANY(\"" + "\", \"".join(category_filters) + "\")"
            filter_parts.append(category_filter_string)
        tenant_filters = filter_facets.get('tenant', [])
        if tenant_filters:
            tenant_filter_string = "tenant: ANY(\"" + "\", \"".join(tenant_filters) + "\")"
            filter_parts.append(tenant_filter_string)
    if filter_tenant:
        filter_tenant_string = "tenant: ANY(\"" + filter_tenant + "\")"
        filter_parts.append(filter_tenant_string)
    filter_string = " AND ".join(filter_parts)

    ## Request
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        page_token=page_token,
        query=search_query,
        page_size=page_size,
        content_search_spec=content_search_spec,
        facet_specs=facet_specs,
        filter=filter_string,
    )

    response = client.search(request)
    # print("~ API Response:\n",response)

    formatted_results = []
    results = response.results
    numOfResults = len(results)
    totalSize = response.total_size
    attributionToken = response.attribution_token
    nextPageToken = response.next_page_token
    summary = response.summary.summary_text
    corrected_query = response.corrected_query
    facets = response.facets
    facetDict = {}
    for facet in facets:
        key = facet.key
        values_dict = {value.value: value.count for value in facet.values}
        facetDict[key] = values_dict

    some_results = {
      "numOfResults": numOfResults,
      "totalSize": totalSize,
      "token": attributionToken,
      "nextPageToken": nextPageToken,
      "summary": summary,
      "corrected_query": corrected_query,
      "facets": facetDict
    }

    formatted_results.append(some_results)

    for result in response.results:
      data = MessageToDict(result.document._pb)
      formatted_result = {}
      formatted_result['id'] = data.get('id', {})
      formatted_result['filter_category'] = data.get('structData', {}).get('category',{})
      formatted_result['filter_id'] = data.get('structData', {}).get('id',{})
      formatted_result['filter_name'] = data.get('structData', {}).get('name',{})
      formatted_result['filter_tenant'] = data.get('structData', {}).get('tenant',{})
      formatted_result['filter_summary'] = data.get('structData', {}).get('summary',{})
      formatted_result['filter_num_pages'] = data.get('structData', {}).get('num_pages',{})
      formatted_result['filter_created_time'] = data.get('structData', {}).get('created_time',{})

      formatted_result['snippets'] = [d.get('snippet') for d in data.get('derivedStructData', {}).get('snippets', []) if d.get('snippet') is not None]
      formatted_result['pageNumber'] = [d.get('pageNumber') for d in data.get('derivedStructData', {}).get('snippets', []) if d.get('pageNumber') is not None]
      formatted_result['uri_link'] = data.get('derivedStructData', {}).get('link',{})
      formatted_result['extractive_answers_content'] = [d.get('content') for d in data.get('derivedStructData', {}).get('extractive_answers', []) if d.get('content') is not None]
      formatted_result['extractive_answers_pageNumber'] = [d.get('pageNumber') for d in data.get('derivedStructData', {}).get('extractive_answers', []) if d.get('pageNumber') is not None]
      formatted_result['extractive_segments'] = [d.get('content') for d in data.get('derivedStructData', {}).get('extractive_segments', []) if d.get('content') is not None]

      formatted_results.append(formatted_result)

    # print("~ Formatted Results:\n",formatted_results)
    # return response, formatted_results
    return formatted_results


In [ ]:
#@title Run Search

project_id = "elroy-demo" #@param {type: "string"}
engine_id = "pca_1693716174825" #@param {type: "string"}
page_token = ""
search_query = "cloudsql vs bigquery" #@param {type: "string"}
page_size = 3 #@param {type: "integer"}
summary_result_count = 1 #@param {type: "integer"}
max_snippet_count = 1 #@param {type: "integer"}
max_extractive_answer_count = 2 #@param {type: "integer"}
max_extractive_segment_count = 2 #@param {type: "integer"}
filter_id = []
filter_facets = []
filter_tenant = ""

request = {
    "project_id": project_id,
    "engine_id": engine_id,
    "page_token": page_token,
    "search_query": search_query,
    "page_size": page_size,
    "summary_result_count": summary_result_count,
    "max_snippet_count": max_snippet_count,
    "max_extractive_answer_count": max_extractive_answer_count,
    "max_extractive_segment_count": max_extractive_segment_count,
    "filter_id": filter_id,
    "filter_facets": filter_facets,
    "filter_tenant": filter_tenant
}

response = search(
    request["project_id"],
    request["engine_id"],
    request["page_token"],
    request["search_query"],
    request["page_size"],
    request["summary_result_count"],
    request["max_snippet_count"],
    request["max_extractive_answer_count"],
    request["max_extractive_segment_count"],
    request["filter_id"],
    request["filter_facets"],
    request["filter_tenant"],
)

response

[{'numOfResults': 3,
  'totalSize': 3847,
  'token': 'W_BaCgwIgYbeqQYQ_qbTzwISJDY1MzJlZDk5LTAwMDAtMmM1MS05YzZjLTE0MjIzYmFiODkyYSIHR0VORVJJQyoc1LKdFZyGjiKiho4ijr6dFaaL7xfFy_MXwvCeFQ',
  'nextPageToken': 'QYykDOiFmYzIjM0ETLjZzY50SM1MmMtADMwATL4kDZlJzM1YDJaIA0hv9_QYQqt_egIwgEzEgC',
  'summary': 'BigQuery is a good option for meeting the requirement to store at least 10 TB of historical data [1].',
  'corrected_query': '',
  'facets': {'category': {'pca': 7843}, 'tenant': {'gcp': 7843}}},
 {'id': 'Official-Google-Cloud-Certified-Professi_20230827171845',
  'filter_category': 'pca',
  'filter_id': 'Official-Google-Cloud-Certified-Professi_20230827171845',
  'filter_name': 'Official Google Cloud Certified Professional Cloud Architect Study Guide',
  'filter_tenant': 'gcp',
  'filter_summary': {},
  'filter_num_pages': {},
  'filter_created_time': {},
  'snippets': ['Designing for Technical Requirements Fully managed and serverless databases, such as Cloud Datastore and <b>BigQuery</b> ... <b

## Vertex AI

In [ ]:
#@title VertexAI

! pip install google-cloud-aiplatform --quiet

import re
import os
import vertexai
from vertexai.preview.language_models import TextGenerationModel


def vertex_qa(
  project_id,
  llmModel,
  query,
  summary,
  searchResults,
  userInput,
  temperature,
  topK,
  topP
  ):

  ######################################################################################################

  """ Snippet / Answer / Segment """
  snippet_pattern = r"\{\{snippet_(\d+)\}\}"
  answer_pattern = r"\{\{answer_(\d+)-(\d+)\}\}"
  segment_pattern = r"\{\{segment_(\d+)-(\d+)\}\}"
  doc_pattern = r"\{\{docName_(\d+)\}\}"

  # Find all occurrences of each item in the userInput
  snippets = re.findall(snippet_pattern, userInput)
  answers = re.findall(answer_pattern, userInput)
  segments = re.findall(segment_pattern, userInput)
  docs = re.findall(doc_pattern, userInput)

  # Create a dictionary to store the key-value pairs
  result_dict = {}

  # Process snippet items and store in the result_dict
  for snippet_index in snippets:
    snippet_index = int(snippet_index)
    if snippet_index <= len(searchResults):
      # print("Snippet:",snippet_index)
      value = searchResults[snippet_index - 1]["snippets"][0]
    else:
      value = ""
    key = f"{{snippet_{snippet_index}}}"
    result_dict[key] = value

  # Process answer items and store in the result_dict
  for answer_index_1, answer_index_2 in answers:
    answer_index_1, answer_index_2 = int(answer_index_1), int(answer_index_2)
    if answer_index_1 <= len(searchResults) and answer_index_2 <= len(searchResults[answer_index_1 - 1]["extractive_answers_content"]):
      # print("Answer:", answer_index_1, answer_index_2)
      value = searchResults[answer_index_1 - 1]["extractive_answers_content"][answer_index_2 - 1]
    else:
      value = ""
    key = f"{{answer_{answer_index_1}-{answer_index_2}}}"
    result_dict[key] = value

  # Process answer items and store in the result_dict
  for segment_index_1, segment_index_2 in segments:
    segment_index_1, segment_index_2 = int(segment_index_1), int(segment_index_2)
    if segment_index_1 <= len(searchResults) and segment_index_2 <= len(searchResults[segment_index_1 - 1]["extractive_segments"]):
      # print("Segment:", segment_index_1, segment_index_2)
      value = searchResults[segment_index_1 - 1]["extractive_segments"][segment_index_2 - 1]
    else:
      value = ""
    key = f"{{segment_{segment_index_1}-{segment_index_2}}}"
    result_dict[key] = value

  # Process doc items and store in the result_dict
  for doc_index in docs:
    doc_index = int(doc_index)
    if doc_index <= len(searchResults):
      # print("Doc:", doc_index)
      value = searchResults[doc_index - 1]["filter_name"]
    else:
      value = ""
    key = f"{{docName_{doc_index}}}"
    result_dict[key] = value

  # Process other {{}} and store in the result_dict
  result_dict["{{query}}"] = query
  result_dict["{{summary}}"] = summary

  ######################################################################################################

  """ Vertex AI """

  # https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#foundation_models
  max_output_tokens = 1024 # text-bison
  if llmModel=="text-bison-32k":
    max_output_tokens = 8192 # max output for text-bison-32k

  temperature = 0.0 if temperature<0.0 else temperature
  temperature = 1.0 if temperature>1.0 else temperature
  topK = 1 if topK<1 else topK
  topK = 40 if topK>40 else topK
  topP = 0.0 if topP<0.0 else topP
  topP = 1.0 if topP>1.0 else topP

  # llm = VertexAI(
  #   model_name=llmModel,
  #   max_output_tokens=max_output_tokens,
  #   temperature=temperature,
  #   top_k=topK,
  #   top_p=topP,
  #   verbose=True)

  ## Replace each {x} with its actual value
  for i in result_dict:
    userInput = userInput.replace(i, result_dict[i])

  # result = llm(userInput)
  result = predict_large_language_model_sample(project_id, llmModel, temperature, topP, topK, userInput)


  # print("Final Prompt:\n", userInput)
  # print("-----------")
  print("LLM Output:\n", result)

  return result

def predict_large_language_model_sample(
    project_id: str,
    model_name: str,
    temperature: float,
    top_p: float,
    top_k: int,
    content: str,
    max_decode_steps: int=256,
    location: str = "us-central1",
    tuned_model_name: str = "",
    ) :
    """Predict using a Large Language Model."""
    vertexai.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained(model_name)
    if tuned_model_name:
      model = model.get_tuned_model(tuned_model_name)
    response = model.predict(
        content,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)

    # print(f"Response from Model: {response.text}")
    return response.text

In [ ]:
#@title Run VertexAI

project_id = "elroy-demo" #@param {type: "string"}
llmModel = "text-bison" #@param {type: "string"}
query = search_query
summary = response[0]['summary']
searchResults = response[1:]
prompt = "Using only the given context, provide a summary answer for query and structure your answer in a markdown table format." #@param {type: "string"}
temperature = 0
topK = 40
topP = 0.95

ans = vertex_qa(
  project_id,
  llmModel,
  query,
  summary,
  searchResults,
  prompt,
  temperature,
  topK,
  topP
)


LLM Output:
  | Question | Answer |
| ----------- | ----------- |
| What is the main idea of the passage? | The main idea of the passage is that using only the given context, one can provide a summary answer for a query and structure the answer in a markdown table format. |
| What are the steps involved in providing a summary answer for a query? | The steps involved in providing a summary answer for a query are: 
1. Identify the main idea of the passage. 
2. Structure the answer in a markdown table format. 
3. Include only the given context in the answer. |
